<a href="https://colab.research.google.com/github/thangvle/smart_pros/blob/master/trial_Tri_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import tensorflow as tf
import pylab as pl
import numpy as np
import matplotlib
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
# need the following 2 libraries to read csv files
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt



# importing EMG data
  # first download the csv file to tensorflow & label it
!wget -O sheetnamedude.csv https://raw.githubusercontent.com/thangvle/smart_pros/master/EMG_label_data_muscle1.csv
  # now we can use that name to read the data
data = pd.read_csv("sheetnamedude.csv")

# turn time and voltage to 1D array
data.columns = ["time_stamp", "voltage", "time", "label"]
time = data["time"].values
voltage = data["voltage"].values
labels = data["label"].values
print("Time")
print(time)
print("voltage")
print(voltage)
print("label")
print(labels)

train_data, test_data = train_test_split(data, train_size=0.8)
print("train_data")
print(train_data)
print("test_data")
print(test_data)


"""
TODO

- Initialize variable for time and voltage as 1D array
- Initialize Weight function
- Create loss function and optimizer (SGD)
- tf.session
"""

def conv_relu(inputs, filters, k_size, stride, padding, scope_name):

    # method that does convolution + relu on inputs

    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_channels = inputs.shape[-1]
        kernel = tf.get_variable('kernel',
                                [k_size, k_size, in_channels, filters],
                                initializer=tf.truncated_normal_initializer())
        biases = tf.get_variable('biases',
                                [filters],
                                initializer=tf.random_normal_initializer())
        conv = tf.nn.conv1d(inputs, kernel, strides=[1, stride, stride, 1], padding=padding)
    return tf.nn.relu(conv + biases, name=scope.name)

def maxpool(inputs, ksize, stride, padding='VALID', scope_name='pool'):
    '''A method that does max pooling on inputs'''
    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        pool = tf.nn.max_pool(inputs,
                            ksize=[1, ksize, ksize, 1],
                            strides=[1, stride, stride, 1],
                            padding=padding)
    return pool

def fully_connected(inputs, out_dim, scope_name='fc'):
    # A fully connected linear layer on inputs

    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_dim = inputs.shape[-1]
        w = tf.get_variable('weights', [in_dim, out_dim],
                            initializer=tf.truncated_normal_initializer())
        b = tf.get_variable('biases', [out_dim],
                            initializer=tf.constant_initializer(0.0))
        out = tf.matmul(inputs, w) + b
    return out


class ConvNet(object):
  def __init__(self):
	     self.learing_rate = 0.001		# set up learning rate
  self.batch_size = 300			# split up data in 1 epoch
self.keep_prob = tf.constant(0.75)	# probablity
self.gstep = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
self.n_classes = 10
self.skip_step = 20
self.n_test = 10000
self.training = False
def get_data(self):
	       with tf.name_scope('data'):
               # need to have label with the train_data and test_data
       	    # create dataset
       		   train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000)
train_data = train_data.batch(batch_size)

test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

# creating iterator
iterator = tf.data.Iterator.from_structure(train_data.output_typestrain_data.output_shapes)
# reshape array here if needed

# Initializer for train and test Dataset
self.train_init = iterator.make_initializer(train_data)
self.test_init = iterator.make_initializer(test_data)

def inference(self):
        conv1 = conv_relu(inputs=self.img,
                        filters=32,
                        k_size=5,
                        stride=1,
                        padding='SAME',
                        scope_name='conv1')
        pool1 = maxpool(conv1, 2, 2, 'VALID', 'pool1')

        fc = fully_connected(pool1, 1024, 'fc')
        dropout = tf.nn.dropout(tf.nn.relu(fc), self.keep_prob, name='relu_dropout')
        self.logits = fully_connected(dropout, self.n_classes, 'logits')

def loss(self):
        '''
        define loss function
        use softmax cross entropy with logits as the loss function
        compute mean cross entropy, softmax is applied internally
        '''
        #
        with tf.name_scope('loss'):
            entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.label, logits=self.logits)
            self.loss = tf.reduce_mean(entropy, name='loss')

def optimize(self):
    '''
    Define training op
    using Adam Gradient Descent to minimize cost
    '''
    self.opt = tf.train.AdamOptimizer(self.lr).minimize(self.loss,
                                            global_step=self.gstep)

def summary(self):
    '''
    Create summaries to write on TensorBoard
    '''
    with tf.name_scope('summaries'):
        tf.summary.scalar('loss', self.loss)
        tf.summary.scalar('accuracy', self.accuracy)
        tf.summary.histogram('histogram loss', self.loss)
        self.summary_op = tf.summary.merge_all()

def eval(self):
    '''
    Count the number of right predictions in a batch
    '''
    with tf.name_scope('predict'):
        preds = tf.nn.softmax(self.logits)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(self.label, 1))
        self.accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

def build(self):
    '''
    Build the computation graph
    '''
    self.get_data()
    self.inference()
    self.loss()
    self.optimize()
    self.eval()
    self.summary()

def train_one_epoch(self, sess, saver, init, writer, epoch, step):
    start_time = time.time()
    sess.run(init)
    self.training = True
    total_loss = 0
    n_batches = 0
    try:
        while True:
            _, l, summaries = sess.run([self.opt, self.loss, self.summary_op])
            writer.add_summary(summaries, global_step=step)
            if (step + 1) % self.skip_step == 0:
                print('Loss at step {0}: {1}'.format(step, l))
            step += 1
            total_loss += l
            n_batches += 1
    except tf.errors.OutOfRangeError:
        pass
    saver.save(sess, 'checkpoints/convnet_mnist/mnist-convnet', step)
    print('Average loss at epoch {0}: {1}'.format(epoch, total_loss/n_batches))
    print('Took: {0} seconds'.format(time.time() - start_time))
    return step

def eval_once(self, sess, init, writer, epoch, step):
    start_time = time.time()
    sess.run(init)
    self.training = False
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch, summaries = sess.run([self.accuracy, self.summary_op])
            writer.add_summary(summaries, global_step=step)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy at epoch {0}: {1} '.format(epoch, total_correct_preds/self.n_test))
    print('Took: {0} seconds'.format(time.time() - start_time))

def train(self, n_epochs):
    '''
    The train function alternates between training one epoch and evaluating
    '''
    utils.safe_mkdir('checkpoints')
    utils.safe_mkdir('checkpoints/convnet_mnist')
    writer = tf.summary.FileWriter('./graphs/convnet', tf.get_default_graph())

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_mnist/checkpoint'))
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)

        step = self.gstep.eval()

        for epoch in range(n_epochs):
            step = self.train_one_epoch(sess, saver, self.train_init, writer, epoch, step)
            self.eval_once(sess, self.test_init, writer, epoch, step)
    writer.close()

if __name__ == '__main__':
    model = ConvNet()
    model.build()
    model.train(n_epochs=30)

--2019-03-24 01:28:05--  https://raw.githubusercontent.com/thangvle/smart_pros/master/EMG_label_data_muscle1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11285 (11K) [text/plain]
Saving to: ‘sheetnamedude.csv’

sheetnamedude.csv   100%[===================>]  11.02K  --.-KB/s    in 0s      

2019-03-24 01:28:05 (107 MB/s) - ‘sheetnamedude.csv’ saved [11285/11285]

Time
[3010 3020 3032 3042 3052 3063 3074 3085 3095 3105 3117 3127 3137 3148
 3159 3170 3180 3190 3202 3212 3222 3233 3244 3254 3265 3275 3287 3297
 3307 3318 3329 3339 3350 3360 3371 3382 3392 3403 3414 3424 3435 3445
 3456 3467 3477 3488 3499 3509 3520 3530 3540 3552 3562 3573 3584 3594
 3605 3615 3625 3637 3647 3658 3668 3679 3690 3700 3710 3722 3732 3743
 3753 3764 3775 3785 3795 3807 3817 3828

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


NameError: ignored